1. Khai báo thư viện

In [1]:
import pandas as pd
from textblob import TextBlob
import pycountry

2. Load file data

In [2]:
data_file_path = "D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\sentimentdataset.csv"
data = pd.read_csv(data_file_path)
df = pd.DataFrame(data)

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HaoPham\AppData\Local\Temp\ipykernel_27720\2787106789.py:1: SyntaxWarning: invalid escape sequence '\D'
  data_file_path = "D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\sentimentdataset.csv"


3. Tạo các bảng Dim

3.1 Tạo bảng Users

In [3]:
# Tạo bảng Users và phân loại các users
df['Total_Interaction'] = df['Retweets'] + df['Likes']
# Phân loại các user
df['User Type'] = df['Total_Interaction'].apply(lambda x: 'Influencer' if x > 50 else 'Regular User')

df['User ID'] = ['User_' + str(i+1) for i in range(len(df))]  # Tạo User IDs

# Tính số bài viết của mỗi user (Posts_count)
user_posts_count = df.groupby('User')['Text'].count().reset_index()
user_posts_count.columns = ['User', 'Posts_count']

df_users = df[['User ID', 'User', 'User Type']].drop_duplicates().merge(user_posts_count, on='User')
df_users.columns = ['User ID', 'User Name', 'User Type', 'Posts_count']

In [4]:
df_users.head()

,User ID,User Name,User Type,Posts_count
0,User_1,User123,Regular User,1
1,User_2,CommuterX,Regular User,1
2,User_3,FitnessFan,Influencer,1
3,User_4,AdventureX,Regular User,1
4,User_5,ChefCook,Regular User,1


3.2 Tạo bảng Coutries

In [5]:
# Tạo bảng Countries với mã ISO 3166-1 alpha-2
def get_country_code(country_name):
    if country_name == "UK":
        return "UK"
    elif country_name == "Scotland":
        return "SCT"
    try:
        return pycountry.countries.lookup(country_name).alpha_2
    except LookupError:
        return None  # Trả về None nếu không tìm thấy

# Áp dụng hàm để tạo cột 'Country ID'
df['Country ID'] = df['Country'].apply(get_country_code)

# Tạo bảng df_countries với các giá trị duy nhất
df_countries = df[['Country', 'Country ID']].drop_duplicates()

In [6]:
df_countries.head()

,Country,Country ID
0,USA,US
1,Canada,CA
3,UK,UK
4,Australia,AU
5,India,IN


3.3 Tạo bảng Platform

In [7]:
#Tạo bảng Dim Platform
df_platform = pd.DataFrame({
    'Platform ID': ['FB', 'IG', 'X'],
    'Platform': ['Facebook', 'Instagram', 'Twitter']
})

In [8]:
df_platform.head()

,Platform ID,Platform
0,FB,Facebook
1,IG,Instagram
2,X,Twitter


3.4 Tạo bảng Calendar

In [9]:
#Tạo bảng Calendar
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df_calendar = df[['Timestamp']].drop_duplicates()
df_calendar['Year'] = df_calendar['Timestamp'].dt.year
df_calendar['Month'] = df_calendar['Timestamp'].dt.month
df_calendar['Day'] = df_calendar['Timestamp'].dt.day
df_calendar['Hour'] = df_calendar['Timestamp'].dt.hour

In [10]:
df_calendar.head()

,Timestamp,Year,Month,Day,Hour
0,2023-01-15 12:30:00,2023,1,15,12
1,2023-01-15 08:45:00,2023,1,15,8
2,2023-01-15 15:45:00,2023,1,15,15
3,2023-01-15 18:20:00,2023,1,15,18
4,2023-01-15 19:55:00,2023,1,15,19


-------CÁC BẢNG ĐẶC BIỆT-------

3.5 Tạo bảng Sentiments

In [11]:
#Tạo bảng Sentiments
def classify_sentiment(text):
    if text.lower() == 'neutral':
        return 'Neutral'
    else:
        analysis = TextBlob(text)
        polarity = analysis.sentiment.polarity
        return 'Positive' if polarity > 0 else 'Negative'

df['Classified Sentiment'] = df['Sentiment'].apply(classify_sentiment)

df_sentiments = df[['Sentiment', 'Classified Sentiment']].drop_duplicates()
df_sentiments.columns = ['Sentiments', 'Classified Sentiment']

In [12]:
df_sentiments.head()

,Sentiments,Classified Sentiment
0,Positive,Positive
1,Negative,Negative
4,Neutral,Neutral
52,Anger,Negative
53,Fear,Negative


3.6 Tạo bảng Hashtags

In [13]:
hashtags_series = df['Hashtags'].str.split().explode()
hashtags_counts = hashtags_series.value_counts().reset_index()
hashtags_counts.columns = ['Hashtags', 'Hashtags Count']

df_hashtags = hashtags_counts[['Hashtags', 'Hashtags Count']]

In [14]:
df_hashtags.head()

,Hashtags,Hashtags Count
0,#Serenity,15
1,#Gratitude,13
2,#Excitement,13
3,#Despair,11
4,#Nostalgia,11


4. Tạo bảng Fact Tweets

In [15]:
# Map giá trị Platform thành Platform ID
df['Platform ID'] = df['Platform'].map(df_platform.set_index('Platform')['Platform ID'])

# Tạo bảng Fact Tweets và thay Platform bằng Platform ID đã map
df_tweets = df[['Text', 'Classified Sentiment', 'Timestamp', 'User ID', 'Platform ID', 'Hashtags', 'Retweets', 'Likes', 'Country ID']]
df_tweets.columns = ['Text', 'Sentiment', 'Timestamp', 'User ID', 'Platform ID', 'Hashtags', 'Retweets', 'Likes', 'Country ID']
# Thêm cột ID cho bảng df_tweets
df_tweets['Tweet ID'] = 'Posts_' + (df_tweets.index + 1).astype(str)
df_tweets = df_tweets[['Tweet ID', 'Text', 'Sentiment', 'Timestamp', 'User ID', 'Platform ID', 'Hashtags', 'Retweets', 'Likes', 'Country ID']]

C:\Users\HaoPham\AppData\Local\Temp\ipykernel_27720\1618338067.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets['Tweet ID'] = 'Posts_' + (df_tweets.index + 1).astype(str)


In [16]:
df_tweets.head()

,Tweet ID,Text,Sentiment,Timestamp,User ID,Platform ID,Hashtags,Retweets,Likes,Country ID
0,Posts_1,Enjoying a beautiful day at the park,Positive,2023-01-15 12:30:00,User_1,X,#Nature #Park,15,30,US
1,Posts_2,Traffic was terrible this morning,Negative,2023-01-15 08:45:00,User_2,X,#Traffic #Morning,5,10,CA
2,Posts_3,Just finished an amazing workout,Positive,2023-01-15 15:45:00,User_3,IG,#Fitness #Workout,20,40,US
3,Posts_4,Excited about the upcoming weekend getaway,Positive,2023-01-15 18:20:00,User_4,FB,#Travel #Adventure,8,15,UK
4,Posts_5,Trying out a new recipe for dinner tonight,Neutral,2023-01-15 19:55:00,User_5,IG,#Cooking #Food,12,25,AU


4.1 Tạo bảng liên kết bảng Hashtag vs bảng Tweets

In [17]:
# Bước 1: Tách các hashtag theo khoảng trắng và sử dụng explode để tách thành nhiều hàng
hashtags_series = df_tweets[['Tweet ID', 'Hashtags']].copy()  # Lấy bản sao của cột Tweet ID và Hashtags
hashtags_series['Hashtags'] = hashtags_series['Hashtags'].str.split(' ')  # Tách hashtags theo khoảng trắng
hashtags_series = hashtags_series.explode('Hashtags')  # Tạo các hàng riêng cho mỗi hashtag
hashtags_series = hashtags_series.reset_index(drop=True) # Xử lí lại index cho bảng

df_hashtag_links = pd.DataFrame(hashtags_series)

In [18]:
df_hashtag_links.head()

,Tweet ID,Hashtags
0,Posts_1,#Nature
1,Posts_1,#Park
2,Posts_2,#Traffic
3,Posts_2,#Morning
4,Posts_3,#Fitness


5. Lưu các bảng dữ liệu đã tách

In [19]:
# Đọc dữ liệu gốc
df.to_csv('D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\sentimentdataset03.csv')

# Thực hiện các bước tạo các bảng và Fact như đã hướng dẫn ở trên

# Lưu bảng Users
df_users.to_csv(r'D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\users.csv', index=False)

# Lưu bảng Sentiments
df_sentiments.to_csv('D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\sentiments.csv', index=False)

# Lưu bảng Hashtags
df_hashtags.to_csv('D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\hashtags.csv', index=False)

# Lưu bảng Hashtags links
df_hashtag_links.to_csv('D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\hashtag_links.csv', index=False)

# Lưu bảng Countries
df_countries.to_csv('D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\countries.csv', index=False)

# Lưu bảng Calendar
df_calendar.to_csv('D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\calendar.csv', index=False)

# Lưu bảng Platform
df_platform.to_csv('D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\platform.csv', index=False)

# Lưu bảng Tweets
df_tweets.to_csv('D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data\cleaned_data\Tweets.csv', index=False)

# In thông báo sau khi lưu xong
print("Các bảng đã được lưu thành công!")


Các bảng đã được lưu thành công!


<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:10: SyntaxWarning: invalid escape sequence '\D'
<>:13: SyntaxWarning: invalid escape sequence '\D'
<>:16: SyntaxWarning: invalid escape sequence '\D'
<>:19: SyntaxWarning: invalid escape sequence '\D'
<>:22: SyntaxWarning: invalid escape sequence '\D'
<>:25: SyntaxWarning: invalid escape sequence '\D'
<>:28: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:10: SyntaxWarning: invalid escape sequence '\D'
<>:13: SyntaxWarning: invalid escape sequence '\D'
<>:16: SyntaxWarning: invalid escape sequence '\D'
<>:19: SyntaxWarning: invalid escape sequence '\D'
<>:22: SyntaxWarning: invalid escape sequence '\D'
<>:25: SyntaxWarning: invalid escape sequence '\D'
<>:28: SyntaxWarning: invalid escape sequence '\D'
C:\Users\HaoPham\AppData\Local\Temp\ipykernel_27720\1721402155.py:2: SyntaxWarning: invalid escape sequence '\D'
  df.to_csv('D:\DATN\Social Media Sentiments Analysis Dataset\DeBug  data